<small><font color=gray>Notebook author: <a href="https://www.linkedin.com/in/olegmelnikov/" target="_blank">Oleg Melnikov</a> ©2021 onwards</font></small><hr style="margin:0;background-color:silver">

**[<font size=6>🌌Stellar</font>](https://www.kaggle.com/competitions/2oct23hse-stellar/rules)**. [**Instructions**](https://colab.research.google.com/drive/1riOGrE_Fv-yfIbM5V4pgJx4DWcd92cZr#scrollTo=ITaPDPIQEgXV) for running Colabs.

<small>**(Optional) CONSENT.** <mark>[ X ]</mark> We consent to sharing our Colab (after the assignment ends) with other students/instructors for educational purposes. We understand that sharing is optional and this decision will not affect our grade in any way. <font color=gray><i>(If ok with sharing your Colab for educational purposes, leave "X" in the check box.)</i></font></small>

In [ ]:
#from google.colab import drive; drive.mount('/content/drive')   # OK to enable, if your kaggle.json is stored in Google Drive

In [ ]:
!pip -q install --upgrade --force-reinstall --no-deps kaggle > log  # upgrade kaggle package (to avoid a warning)
!mkdir -p ~/.kaggle                                           # .kaggle folder must contain kaggle.json for kaggle executable to properly authenticate you to Kaggle.com
#!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json >log  # First, download kaggle.json from kaggle.com (in Account page) and place it in the root of mounted Google Drive
!cp kaggle.json ~/.kaggle/kaggle.json > log                   # Alternative location of kaggle.json (without a connection to Google Drive)
!chmod 600 ~/.kaggle/kaggle.json                              # give only the owner full read/write access to kaggle.json
!kaggle config set -n competition -v 2oct23hse-stellar        # set the competition context for the next few kaggle API calls. !kaggle config view - shows current settings
!kaggle competitions download >> log                          # download competition dataset as a zip file
!unzip -o *.zip >> log                                        # Kaggle dataset is copied as a single file and needs to be unzipped.
lb =!kaggle competitions leaderboard --show                   # print public leaderboard

- competition is now set to: 2oct23hse-stellar
100% 7.58M/7.58M [00:00<00:00, 43.2MB/s]


In [ ]:
%%time
%%capture
%reset -f
from IPython.core.interactiveshell import InteractiveShell as IS; IS.ast_node_interactivity = "all"
import numpy as np, pandas as pd, time, matplotlib.pyplot as plt, seaborn as sns, os, tqdm, re, sys, cv2, skimage
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression as LR
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA, LinearDiscriminantAnalysis as LDA
ToCSV = lambda df, fname: df.round(2).to_csv(f'{fname}.csv', index_label='id') # rounds values to 2 decimals

class Timer():
  def __init__(self, lim:'RunTimeLimit'=60): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

np.set_printoptions(linewidth=100, precision=2, edgeitems=2, suppress=True)
pd.set_option('display.max_columns', 20, 'display.precision', 2, 'display.max_rows', 4)

CPU times: user 1.39 s, sys: 191 ms, total: 1.58 s
Wall time: 2.67 s


In [ ]:
df = pd.read_csv('XY_Stellar.csv'); df

,alpha,delta,u,g,r,i,z,run_ID,cam_col,field_ID,redshift,plate,MJD,fiber_ID,Class
0,11.64,21.28,26.28,26.15,24.05,18.87,19.00,8848,5,272,0.84,7740,56824,833,NaN
1,173.09,42.21,22.51,22.83,22.21,19.55,19.96,4156,3,486,0.81,9041,58067,428,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199998,131.31,44.27,24.07,24.64,21.63,19.20,19.03,7076,3,251,0.55,6014,56166,1021,G
199999,22.59,0.25,25.30,25.56,24.09,19.41,19.96,5164,4,511,1.26,9590,57969,878,G


In [ ]:
df.info()   # observe datatypes and any missing values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 15 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   alpha     200000 non-null  float64
 1   delta     200000 non-null  float64
 2   u         200000 non-null  float64
 3   g         200000 non-null  float64
 4   r         200000 non-null  float64
 5   i         200000 non-null  float64
 6   z         200000 non-null  float64
 7   run_ID    200000 non-null  int64  
 8   cam_col   200000 non-null  int64  
 9   field_ID  200000 non-null  int64  
 10  redshift  200000 non-null  float64
 11  plate     200000 non-null  int64  
 12  MJD       200000 non-null  int64  
 13  fiber_ID  200000 non-null  int64  
 14  Class     160000 non-null  object 
dtypes: float64(8), int64(6), object(1)
memory usage: 22.9+ MB


In [ ]:
# Change string labels to numbers in order of increasing size of the entity (Star < Quasi Star < Galaxy)
#df.Class = df.Class.apply(lambda C: -1 if C=='S' else 0 if C=='Q' else 1 if C=='G' else None)

In [ ]:
vX = df.query('Class!=Class').drop('Class', axis=1)  # slice a test sample
tXY = df.query('Class==Class')                       # slice training sample
tX, tY = tXY.drop('Class', axis=1), tXY.Class        # split into training I/O

In [ ]:
def ScatterCorrHist(df):
  def corrdot(*args, **kwargs):
    # credit: https://stackoverflow.com/questions/48139899
    corr_r = args[0].corr(args[1], 'pearson')
    corr_text = f"{corr_r:2.2f}".replace("0.", ".")
    ax = plt.gca();
    ax.set_axis_off();
    msz = abs(corr_r) * 5000   # marker size
    fsz = abs(corr_r) * 40 + 5 # font size
    ax.scatter([.5], [.5], msz, [corr_r], alpha=0.5, cmap='coolwarm', vmin=-1, vmax=1, transform=ax.transAxes)
    ax.annotate(corr_text, [.5, .5,],  xycoords="axes fraction", ha='center', va='center', fontsize=fsz)

  sns.set(style='white', font_scale=.8);
  g = sns.PairGrid(df, aspect=1, diag_sharey=False);
  g.fig.set_size_inches(20,10)
  g.map_lower(sns.regplot, lowess=True, ci=False, line_kws={'color':'red'}, scatter_kws={'s':1});
  g.map_diag(sns.histplot, kde_kws={'color':'black'});
  g.map_upper(corrdot);
  g.fig.suptitle("Scatter plot, Correlations and histograms on diagonal", y=1);
  _ = plt.subplots_adjust(hspace=0.02, wspace=0.02);
  _ = plt.show();

# ScatterCorrHist(tXY.head(200))

In [ ]:
tmr = Timer()

⏳ started. You have 60 sec. Good luck!


<hr color=green size=40>

<strong><font color=green size=5>⏳Timed Green Playground (TGP): Your ideas, code, documentation, and timer START HERE!</font></strong>

<font color=green>Students: Keep all your definitions, code, documentation in <b>TGP</b>. Modifying any code outside of TGP incurs penalties.

In [ ]:
from IPython.core.interactiveshell import InteractiveShell as IS; IS.ast_node_interactivity = "all"
import numpy as np, pandas as pd, time
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LogisticRegression
start = time.time()

def winsorisation(tX_train, tY_train):
    tXY = pd.concat([tX_train, tY_train], axis=1)
    Class_series = tXY['Class']
    tXY.drop('Class', axis=1, inplace=True)

    q_low = tXY['i'].quantile(0.0125)
    q_hi = tXY['i'].quantile(0.995)

    tXY = tXY[(tXY['i'] < q_hi) & (tXY['i'] > q_low)]

    q_low = tXY['r'].quantile(0)
    q_hi = tXY['r'].quantile(1)

    tXY = tXY[(tXY['r'] < q_hi) & (tXY['r'] > q_low)]


    q_low = tXY['field_ID'].quantile(0.025)
    q_hi = tXY['field_ID'].quantile(0.995)

    tXY = tXY[(tXY['field_ID'] < q_hi) & (tXY['field_ID'] > q_low)]

    q_low = tXY['delta'].quantile(0.05)
    q_hi = tXY['delta'].quantile(0.995)

    tXY = tXY[(tXY['delta'] < q_hi) & (tXY['delta'] > q_low)]

    q_low = tXY['MJD'].quantile(0.0105)
    q_hi = tXY['MJD'].quantile(0.99)
    tXY = tXY[(tXY['MJD'] < q_hi) & (tXY['MJD'] > q_low)]

    q_low = tXY['alpha'].quantile(0.05)
    q_hi = tXY['alpha'].quantile(0.99)
    tXY = tXY[(tXY['alpha'] < q_hi) & (tXY['alpha'] > q_low)]

    q_low = tXY['fiber_ID'].quantile(0.0025)
    q_hi = tXY['fiber_ID'].quantile(0.98)
    tXY = tXY[(tXY['fiber_ID'] < q_hi) & (tXY['fiber_ID'] > q_low)]

    tXY = pd.concat([tXY, Class_series], axis=1, join='inner')
    tXY.reset_index(drop=True, inplace=True)

    tX_train, tY_train = tXY.drop('Class', axis=1), tXY.Class
    return tX_train, tY_train

ToCSV = lambda df, fname: df.round(2).to_csv(f'{fname}.csv', index_label='id') # rounds values to 2 decimals

class Timer():
  def __init__(self, lim:'RunTimeLimit'=60): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

np.set_printoptions(linewidth=100, precision=2, edgeitems=2, suppress=True)
pd.set_option('display.max_columns', 20, 'display.precision', 2, 'display.max_rows', 4)


df = pd.read_csv('XY_Stellar.csv')

features_to_drop = ['run_ID', 'u']

df = df.drop(columns=features_to_drop)

column_to_log_transform = 'redshift'

if df[column_to_log_transform].dtype in [float, int]:
    df[column_to_log_transform] = np.log(df[column_to_log_transform] + 0.1)
else:
    print(f"Warning: The '{column_to_log_transform}' column contains non-numeric or missing values.")

df.Class = df.Class.apply(lambda C: -1 if C=='S' else 0 if C=='Q' else 1 if C=='G' else None)
vX = df.query('Class!=Class').drop('Class', axis=1)  # slice a test sample
tXY = df.query('Class==Class')                       # slice training sample

tX, tY = tXY.drop('Class', axis=1), tXY.Class

<font color=green><h3><b>$\alpha$. Build polynomial features</b><h3>

In [ ]:
%%time
np.random.seed(0)
tX0, tY0 = tX.iloc[:30000,:], tY[:30000]   # subsample for model selection experiments
vX0, vY0 = tX.iloc[-30000:,:], tY[-30000:] # subsample for model selection experiments

poly = PolynomialFeatures(degree=2)    # add non-linear features (powers and interactions)
tX0p = poly.fit_transform(tX0.select_dtypes(include=np.number))  # create object on training set
vX0p = poly.transform(vX0.select_dtypes(include=np.number))      # apply the same object to test set
vXp  = poly.transform(vX.select_dtypes(include=np.number))       # apply the same object to test set

CPU times: user 100 ms, sys: 19 ms, total: 119 ms
Wall time: 122 ms


<font color=green><h3><b>$\beta$. Fit a model</b><h3>

In [ ]:
poly = PolynomialFeatures(degree=2)    # add non-linear features (powers and interactions)
tX0p = poly.fit_transform(tX0.select_dtypes(include=np.number))  # create object on training set
vX0p = poly.transform(vX0.select_dtypes(include=np.number))      # apply the same object to test set
vXp  = poly.transform(vX.select_dtypes(include=np.number))       # apply the same object to test set


test_set = df.query('Class!=Class').drop('Class', axis=1)
train_set = df.query('Class==Class')
train_set_train_part, train_set_test_part = train_set.drop('Class', axis=1), train_set.Class

test_size = 0.25
tX_train, tX_test, tY_train, tY_test = train_test_split(tX, tY, test_size=test_size, random_state=42)

tX_train, tY_train = winsorisation(tX_train, tY_train)

poly = PolynomialFeatures(degree=3)
tX_train_poly = poly.fit_transform(tX_train)
tX_test_poly = poly.transform(tX_test)
vX_poly = poly.transform(vX)

scaler = StandardScaler()
tX_train_scaled = scaler.fit_transform(tX_train_poly)
tX_test_scaled = scaler.transform(tX_test_poly)
vX_scaled = scaler.transform(vX_poly)


logreg_model = LogisticRegression(max_iter=300)

logreg_model.fit(tX_train_scaled, tY_train)


test_predictions = logreg_model.predict(tX_test_scaled)
test_accuracy = accuracy_score(tY_test, test_predictions)
print("Accuracy on test set:", test_accuracy)


predicted_labels = logreg_model.predict(vX_scaled)

pY = pd.DataFrame(predicted_labels, index=range(1, len(vX) + 1), columns=['Class'])
pY.to_csv('Logistic_Regression_Predictions.csv', header=True, index_label='Index')

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=300)

Accuracy on test set: 0.96925


<font color=green><h3><b>$\gamma$. Make predictions</b><h3>

In [ ]:
pY = pd.DataFrame(logreg_model.predict(vX_scaled), index=range(1,len(vX)+1), columns=['Class'])  # ensure that labels and observations are in corresponding order
pY.Class = pY.Class.apply(lambda C: 'S' if C==-1 else 'Q' if C==0 else 'G' if C==1 else None)
df.Class.fillna('unknown').value_counts()   # distribution of all train labels
pY.value_counts()                           # distribution of predicted labels
ToCSV(pY, '🌌Baseline🐍')

1.0        95204
unknown    40000
-1.0       34362
0.0        30434
Name: Class, dtype: int64

Class
G        23965
S         8928
Q         7107
dtype: int64

<font color=green><h3><b>$\epsilon$. Documentation</b></h3></font>

<font color=green><h4><b>Task 1. Explain Decisions in Preprocessing Pipeline</b></h4></font>

<font color=green>
Explain elements of your preprocessing pipeline i.e. feature engineering, subsampling, clustering, dimensionality reduction, etc.</font>

<font color=green>

1. Why did you choose these elements? (Something in EDA, prior experience,...? Note: EDA is not required)
1. How do you evaluate the effectiveness of these elements?
1. What else have you tried that worked or didn't?

</font>

Firstly, we estimated models without preprocessing features, but the quality of those models could be improved, so we started analyzing the data more specifically.

We used more empirical approach in preprocessing features. In particular, after estimating different models with and without polynomial features and with various hyperparameters (LDA, QDA, logistic regression), we noticed that logistic regression performed in the better way than another models. Therefore, we evaluated any particular approach by estimating the change in the logistic regression model accuracy.
  
As for the preprocessing mechanisms that we applied, we noticed that removing `run_ID` and `u` from features increased accuracy of the models. Furthermore, we tried taking logarithm of `redshift`, which also influenced our model in a positive way. What is more, we scaled all our features to Normal distribution using Standard Scaler and expressed classes (target variable) in the numeric way by the order of increasing size of the entity (Star < Quasi Star < Galaxy). One more preprocessing step that we used was winsorisation of features in order to redice the effect of outliers in data on the model predictions.

As for the approaches that we decided not to implement, we tried to use upsampling to teach our model to recognize better each class (even the least represented in our data). However, it only reduced the accuracy of our model, so we decided not to include it in our final preprocessing procedure.

<font color=green><h4><b>Task 2. Explain Decisions in Modeling Pipeline</b></h4></font>

<font color=green>
Explain your modeling approach, i.e. ideas you tried and why you thought they would be helpful.

1. How did these decisions guide you in modeling?
1. How do you evaluate the effectiveness of these elements?
1. What else have you tried that worked or didn't?

</font>

Firstly we threw away the ideas of classical linear regression or regression with variable selection as we have deemed them to be to simplistic and non-fitting for our situation. LDA and QDA also proven inefficient when tested, so the only logical option was the logistical regression. During the empirical testing, the best test sample size was determined to be 0.3, however the model still wasn't performing as well as we'd hoped so to boost the performance we decided to test 2 more options: winsorisation and introduction of hyperparameters, the winsorisation was a success as it did show a notable improvement in model performance, however we couldn't finish the hyperparameter idea as they complied for far too long (over 19 hours, it's just that at this mark the code was shut down and the idea scrapped), so we also excluded 2 model parameters: u and run_ID as they also lowered the predicting capability of the model, which was also quite effective at boosting the model performance

<font color=green><h3><b>$\zeta$. References</b></h3></font>

<font color=red><b>Your answer here.</b></font>

<font color=green>
Cite your sources to help your peers learn from these (and to avoid plagiarism claims). At the least, HOML textbook should be cited. Use Google Scholar to draw APA citation format for books and publications. Also cite StackOverflow, package documentation, and other meaningful internet resources.

1. ...
1. ...

<font size=5>⌛</font> <strong><font color=green size=5>Do not exceed competition's runtime limit! Do not write code outside TGP</font></strong>
<hr color=green size=40>

In [ ]:
tmr.ShowTime()    # measure Colab's runtime. Do not remove. Keep as the last cell in your notebook.

Runtime is 100 sec > 60 sec limit!!!


## 💡**Starter Ideas**

1. Tune model hyperparameters
1. Try to linear and non-linear feature normalization: shift/scale, log, divide features by features (investigate scatterplot matrix)
1. Try higher order feature interactions and polynomial features on a small subsample. Then identify key features or select key principal components. The final model can be trained on a larger or even full training sample. You can use [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) to reduce the feature set
1. Do a thorough EDA: look for feature augmentations that result in linear decision boundaries between pairs of classes.
1. Evaluate predictions and focus on poorly predicted "groups":
  1. Strongest missclassifications. E.g. the model is very confident about the wrong label
  1. Evaluate predictions near decision boundaries.
1. Do scatter plots show piecewise linear shape? Can a separate linear model be used on each support, or can the pattern be linearized via transformations?
1. How are date/categorical features treated by the model? Is there a [better way](https://www.google.com/search?q=ways+to+encode+categorical+data) to encode these (perhaps, ordinal) features?
  1. E.g. you could replace codes (or groups of codes) with their frequencies, which may capture the implied "distance" or rarity between category levels.
  1. If encoding ordinal features with integers, should non-equidistant values be considered?
1. Learn astronomy domain and features: [🎦](https://www.youtube.com/results?search_query=Quasi-star), [quasi-star](https://en.wikipedia.org/wiki/Quasi-star), [star](https://en.wikipedia.org/wiki/Star), [galaxy](https://en.wikipedia.org/wiki/Galaxy), [📃](https://arxiv.org/abs/2112.02026)
